In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0  # Normalize the images to [0, 1]
x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Define constants
NOISE_DIM = 100
BATCH_SIZE = 256
EPOCHS = 10000
SAVE_INTERVAL = 1000

In [ ]:
# Generator model
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=NOISE_DIM))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

In [ ]:
# Discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
# Compile GAN models
def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = tf.keras.Input(shape=(NOISE_DIM,))
    gan_output = discriminator(generator(gan_input))
    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return gan

In [ ]:
# Training function
def train_gan(generator, discriminator, gan, epochs, batch_size, save_interval):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_imgs = x_train[idx]
        noise = np.random.normal(0, 1, (batch_size, NOISE_DIM))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, NOISE_DIM))
        g_loss = gan.train_on_batch(noise, valid)

        # Print progress
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        # Save generated images at save_interval
        if epoch % save_interval == 0:
            save_generated_images(generator, epoch)
    # Save the generator model
    generator.save('trained_generator.h5')

In [ ]:
# Function to save generated images
def save_generated_images(generator, epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, NOISE_DIM))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale images to [0, 1]

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(gen_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

In [ ]:
# Build and compile the GAN
generator = build_generator()
discriminator = build_discriminator()
gan = compile_gan(generator, discriminator)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Train the GAN
train_gan(generator, discriminator, gan, EPOCHS, BATCH_SIZE, SAVE_INTERVAL)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Streaming output truncated to the last 5000 lines.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7501 [D loss: 5.224883079528809 | D accuracy: 28.3042311668396] [G loss: 0.0015275138430297375]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
7502 [D loss: 5.22499418258667 | D accuracy: 28.304100036621094] [G loss: 0.001527312328107655]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7503 [D loss: 5.225103855133057 | D accuracy: 28.30459773540497] [G loss: 0.0015271108131855726]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
7504 [D loss: 5.225217819213867 | D accuracy: 28.30485999584198] [G loss: 0.0015269092982634902]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7505 [D loss: 5.225330352783203 | D accuracy: 28.30483615398407] [G loss: 0.0015267078997567296]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
7506 [D loss: 5.225439071655273 | D accuracy: 28.304940462112427] [G loss: 0.001526506501249969]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7507 [D loss: 5.225547790527344 | D accuracy: 28.304892778396606] [G loss: 0.0015263052191585302]
8/8 ━

9999 [D loss: 5.480780601501465 | D accuracy: 28.305435180664062] [G loss: 0.001148822484537959]


In [ ]:
save_generated_images(generator, epoch=30, examples=10, dim=(1, 10), figsize=(10, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [ ]:
pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=0c230c17dca109fe26e65597996dca53162541ed0bc50affc74de80ad8781b01
  Stored in directory: /root/.cache/pip/wheels/15/0c/b3/674aea8c5d91c642c817d4d630bd58faa316724b136844094d
Successfully built utils


In [ ]:
def set_soft_gpu(soft_gpu):
    if soft_gpu:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")


def binary_accuracy(label, pred):
    global _b_acc
    if _b_acc is None:
        _b_acc = tf.keras.metrics.BinaryAccuracy()
    _b_acc.reset_states()
    _b_acc.update_state(label, pred)
    return _b_acc.result()

def save_weights(model):
    name = model.__class__.__name__.lower()
    os.makedirs("./models/{}".format(name), exist_ok=True)
    model.save_weights("./models/{}/model.ckpt".format(name))


## Visuals

def save_gan(model, ep, **kwargs):
    name = model.__class__.__name__.lower()
    if name in ["dcgan", "wgan", "wgangp", "lsgan", "wgandiv", "sagan", "pggan"]:
        imgs = model.call(100, training=False).numpy()
        _save_gan(name, ep, imgs, show_label=False)
    elif name == "gan":
        data = model.call(5, training=False).numpy()
        plt.plot(data.T)
        plt.xticks((), ())
        dir_ = "visual/{}".format(name)
        os.makedirs(dir_, exist_ok=True)
        path = dir_ + "/{}.png".format(ep)
        plt.savefig(path)
    elif name == "cgan" or name == "acgan":
        img_label = np.arange(0, 10).astype(np.int32).repeat(10, axis=0)
        imgs = model.predict(img_label)
        _save_gan(name, ep, imgs, show_label=True)
    elif name in ["infogan"]:
        img_label = np.arange(0, model.label_dim).astype(np.int32).repeat(10, axis=0)
        img_style = np.concatenate(
            [np.linspace(-model.style_scale, model.style_scale, 10)] * 10).reshape((100, 1)).repeat(model.style_dim, axis=1).astype(np.float32)
        img_info = img_label, img_style
        imgs = model.predict(img_info)
        _save_gan(name, ep, imgs, show_label=False)
    elif name in ["ccgan", "pix2pix"]:
        if "img" not in kwargs:
            raise ValueError
        input_img = kwargs["img"][:100]
        mask_width = np.random.randint(model.mask_range[0], model.mask_range[1], len(input_img))
        mask = np.ones(input_img.shape, np.float32)
        for i, w in enumerate(mask_width):
            mask_xy = np.random.randint(0, model.img_shape[0] - w, 2)
            x0, x1 = mask_xy[0], w + mask_xy[0]
            y0, y1 = mask_xy[1], w + mask_xy[1]
            mask[i, x0:x1, y0:y1] = 0
        masked_img = input_img * mask
        imgs = model.predict(masked_img)
        _save_img2img_gan(name, ep, masked_img, imgs)
    elif name == "cyclegan":
        if "img6" not in kwargs or "img9" not in kwargs:
            raise ValueError
        img6, img9 = kwargs["img6"][:50], kwargs["img9"][:50]
        img9_, img6_ = model.g12.call(img6, training=False), model.g21.call(img9, training=False)
        img = np.concatenate((img6.numpy(), img9.numpy()), axis=0)
        imgs = np.concatenate((img9_.numpy(), img6_.numpy()), axis=0)
        _save_img2img_gan(name, ep, img, imgs)
    elif name in ["srgan"]:
        if "img" not in kwargs:
            raise ValueError
        input_img = kwargs["img"][:100]
        imgs = model.predict(input_img)
        _save_img2img_gan(name, ep, input_img, imgs)
    elif name == "stylegan":
        n = 12
        global z1, z2       # z1 row, z2 col
        if "z1" not in globals():
            z1 = np.random.normal(0, 1, size=(n, 1, model.latent_dim))
        if "z2" not in globals():
            z2 = np.random.normal(0, 1, size=(n, 1, model.latent_dim))
        imgs = model.predict([
            np.concatenate(
                (z1.repeat(n, axis=0).repeat(1, axis=1), np.repeat(np.concatenate([z2 for _ in range(n)], axis=0), 2, axis=1)),
                axis=1),
            np.zeros([len(z1)*n, model.img_shape[0], model.img_shape[1]], dtype=np.float32)])
        z1_imgs = -model.predict([z1.repeat(model.n_style, axis=1), np.zeros([len(z1), model.img_shape[0], model.img_shape[1]], dtype=np.float32)])
        z2_imgs = -model.predict([z2.repeat(model.n_style, axis=1), np.zeros([len(z2), model.img_shape[0], model.img_shape[1]], dtype=np.float32)])
        imgs = np.concatenate([z2_imgs, imgs], axis=0)
        rest_imgs = np.concatenate([np.ones([1, 28, 28, 1], dtype=np.float32), z1_imgs], axis=0)
        for i in range(len(rest_imgs)):
            imgs = np.concatenate([imgs[:i*(n+1)], rest_imgs[i:i+1], imgs[i*(n+1):]], axis=0)
        _save_gan(name, ep, imgs, show_label=False, nc=n+1, nr=n+1)
    else:
        raise ValueError(name)
    plt.clf()
    plt.close()


def cvt_gif(folders_or_gan, shrink=10):
    if not isinstance(folders_or_gan, list):
        folders_or_gan = [folders_or_gan.__class__.__name__.lower()]
    for folder in folders_or_gan:
        folder = "visual/"+folder
        fs = [folder+"/" + f for f in os.listdir(folder)]
        imgs = []
        for f in sorted(fs, key=os.path.getmtime):
            if not f.endswith(".png"):
                continue
            try:
                int(os.path.basename(f).split(".")[0])
            except ValueError:
                continue
            img = Image.open(f)
            img = img.resize((img.width//shrink, img.height//shrink), Image.ANTIALIAS)
            imgs.append(img)
        path = "{}/generating.gif".format(folder)
        if os.path.exists(path):
            os.remove(path)
        imgs[-1].save(path, append_images=imgs, optimize=False, save_all=True, duration=400, loop=0)
        print("saved ", path)

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from visual import save_gan, cvt_gif
#from utils import set_soft_gpu, binary_accuracy, save_weights
import numpy as np
import time

In [ ]:

# Load MNIST dataset
def get_mnist_data(batch_size):
    (x_train, _), (_, _) = keras.datasets.mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension
    dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(60000).batch(batch_size)
    return dataset

class GAN(keras.Model):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim

        self.g = self._get_generator()
        self.d = self._get_discriminator()

        # Separate optimizers for generator and discriminator
        self.opt_g = keras.optimizers.Adam(0.0002, beta_1=0.5)
        self.opt_d = keras.optimizers.Adam(0.0002, beta_1=0.5)

        self.loss_func = keras.losses.BinaryCrossentropy(from_logits=True)

    def call(self, n, training=None, mask=None):
        return self.g(tf.random.normal((n, self.latent_dim)), training=training)

    def _get_generator(self):
        model = keras.Sequential([
            keras.layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(self.latent_dim,)),
            keras.layers.BatchNormalization(),
            keras.layers.LeakyReLU(),

            keras.layers.Reshape((7, 7, 256)),
            keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
            keras.layers.LeakyReLU(),

            keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
            keras.layers.LeakyReLU(),

            keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
        ], name="generator")
        return model

    def _get_discriminator(self):
        model = keras.Sequential([
            keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
            keras.layers.LeakyReLU(),
            keras.layers.Dropout(0.3),

            keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
            keras.layers.LeakyReLU(),
            keras.layers.Dropout(0.3),

            keras.layers.Flatten(),
            keras.layers.Dense(1)
        ], name="discriminator")
        return model

    def train_d(self, real_images, fake_images):
        real_labels = tf.ones((real_images.shape[0], 1))
        fake_labels = tf.zeros((fake_images.shape[0], 1))

        with tf.GradientTape() as tape:
            real_predictions = self.d(real_images, training=True)
            fake_predictions = self.d(fake_images, training=True)
            real_loss = self.loss_func(real_labels, real_predictions)
            fake_loss = self.loss_func(fake_labels, fake_predictions)
            loss = real_loss + fake_loss

        grads = tape.gradient(loss, self.d.trainable_variables)
        self.opt_d.apply_gradients(zip(grads, self.d.trainable_variables))  # Use discriminator optimizer
        return loss

    def train_g(self, batch_size):
        noise = tf.random.normal([batch_size, self.latent_dim])
        real_labels = tf.ones((batch_size, 1))  # Fool the discriminator

        with tf.GradientTape() as tape:
            generated_images = self.g(noise, training=True)
            predictions = self.d(generated_images, training=True)
            loss = self.loss_func(real_labels, predictions)

        grads = tape.gradient(loss, self.g.trainable_variables)
        self.opt_g.apply_gradients(zip(grads, self.g.trainable_variables))  # Use generator optimizer
        return loss, generated_images

    def step(self, real_images):
        batch_size = real_images.shape[0]
        g_loss, fake_images = self.train_g(batch_size)
        d_loss = self.train_d(real_images, fake_images)
        return d_loss, g_loss

def train(gan, dataset, epochs):
    for epoch in range(epochs):
        for batch in dataset:
            d_loss, g_loss = gan.step(batch)
        print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss.numpy():.4f} | G Loss: {g_loss.numpy():.4f}")

if __name__ == "__main__":
    LATENT_DIM = 100
    BATCH_SIZE = 128
    EPOCHS = 20

    dataset = get_mnist_data(BATCH_SIZE)
    gan = GAN(LATENT_DIM)
    train(gan, dataset, EPOCHS)


Epoch 1/20 | D Loss: 1.3592 | G Loss: 0.6535
Epoch 2/20 | D Loss: 1.2471 | G Loss: 0.8109
Epoch 3/20 | D Loss: 1.1549 | G Loss: 0.8113
Epoch 4/20 | D Loss: 1.2522 | G Loss: 0.9799
Epoch 5/20 | D Loss: 1.2989 | G Loss: 0.8843
Epoch 6/20 | D Loss: 1.3355 | G Loss: 0.6948
Epoch 7/20 | D Loss: 1.3073 | G Loss: 0.7129
Epoch 8/20 | D Loss: 1.3517 | G Loss: 0.7170
Epoch 9/20 | D Loss: 1.3361 | G Loss: 0.6691
Epoch 10/20 | D Loss: 1.2966 | G Loss: 0.8786
Epoch 11/20 | D Loss: 1.2640 | G Loss: 0.8624
Epoch 12/20 | D Loss: 1.2051 | G Loss: 0.9164
Epoch 13/20 | D Loss: 1.3266 | G Loss: 0.7025
Epoch 14/20 | D Loss: 1.3131 | G Loss: 0.8690
Epoch 15/20 | D Loss: 1.2756 | G Loss: 0.9364
Epoch 16/20 | D Loss: 1.3766 | G Loss: 0.7648
Epoch 17/20 | D Loss: 1.3044 | G Loss: 0.7693
Epoch 18/20 | D Loss: 1.3263 | G Loss: 0.9349
Epoch 19/20 | D Loss: 1.2995 | G Loss: 0.6948
Epoch 20/20 | D Loss: 1.3407 | G Loss: 0.7682


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the MNIST dataset
(train_images, _), (_, _) = keras.datasets.mnist.load_data()
train_images = (train_images.astype('float32') - 127.5) / 127.5  # Normalize to [-1, 1]
train_images = np.expand_dims(train_images, axis=-1)

BUFFER_SIZE = 60000
BATCH_SIZE = 256

# Create the dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Generator Model
def build_generator():
    model = keras.Sequential([
        layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7, 7, 256)),

        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Create generator and discriminator
generator = build_generator()
discriminator = build_discriminator()

# Loss and Optimizers
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Optimizers
generator_optimizer = keras.optimizers.Adam(1e-4)
discriminator_optimizer = keras.optimizers.Adam(1e-4)

# Training Function
EPOCHS = 50
NOISE_DIM = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        generate_and_save_images(generator, epoch + 1, seed)
        generator.save('generator.h5')
        discriminator.save('discriminator.h5')

# Function to generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.savefig(f'generated_epoch_{epoch}.png')
    plt.show()

# Train the model
train(train_dataset, EPOCHS)

# Hyperparameter Tuning Suggestions:
# - Experiment with different learning rates (e.g., 1e-3, 1e-5)
# - Try different batch sizes (e.g., 128, 512)
# - Modify the generator architecture (e.g., more layers, different activation functions)
# - Implement learning rate decay or adaptive optimizers
# - Use different loss functions (e.g., Wasserstein loss)
# - Increase training epochs for better quality outputs


Output hidden; open in https://colab.research.google.com to view.